<a href="https://colab.research.google.com/github/murilo-henrique060/matematica-computacional/blob/main/Atividade%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [313]:
from IPython.display import HTML

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [314]:
class Plot():
  def __init__(self, grid=(1, 1), figsize=(4, 4), axis=(0, 4, 0, 4)):
    self.fig, self.ax = plt.subplots(grid[0], grid[1], figsize=figsize)
    self.fig.tight_layout()

    if grid[0] == 1 and grid[1] == 1:
      ax.grid(True)
      ax.set_xlim(axis[0], axis[1])
      ax.set_ylim(axis[2], axis[3])
      return

    for index, ax in enumerate(self.ax):
      ax.grid(True)
      ax.set_xlim(axis[index][0], axis[index][1])
      ax.set_ylim(axis[index][2], axis[index][3])

  def show(self):
    plt.show()

  def animate(self, update, frames=100, blit=True, interval=100):
    plt.close()
    self.ani = animation.FuncAnimation(self.fig, update, frames=frames, blit=blit, interval=interval)
    return HTML(self.ani.to_html5_video())

In [315]:
class PlotObject():
  def __init__(self, ax, points, **kwargs):
    self.ax = ax
    self.line, = ax.plot((), (), **kwargs)
    self.points = points

  @property
  def points(self):
    return np.array(list(zip(*self.line.get_data())))

  @points.setter
  def points(self, points):
    if len(points) == 0:
      self.line.set_data((), ())
    else:
      self.line.set_data(*zip(*points))

In [316]:
class PointObject(PlotObject):
  def __init__(self, ax, point, color='green', marker='o', **kwargs):
    super().__init__(ax, [point], color=color, marker=marker, **kwargs)

  @property
  def point(self):
    return self.points[0]

  @point.setter
  def point(self, point):
    self.points = [point]

  @property
  def x(self):
    return self.point[0]

  @x.setter
  def x(self, x):
    self.point = (x, self.point[1])

  @property
  def y(self):
    return self.point[1]

  @y.setter
  def y(self, y):
    self.point = (self.point[0], y)

In [317]:
class LineObject(PlotObject):
  def __init__(self, ax, start, end, color='gray', **kwargs):
    super().__init__(ax, [start, end], color=color, **kwargs)

  @property
  def start(self):
    return self.points[0]

  @start.setter
  def start(self, start):
    self.points = [start, self.points[1]]

  @property
  def end(self):
    return self.points[1]

  @end.setter
  def end(self, end):
    self.points = [self.points[0], end]

In [318]:
class FunctionObject(PlotObject):
  def __init__(self, ax, function, samples=100, color='blue', **kwargs):
    self._function = function
    self._samples = samples
    super().__init__(ax, [], color=color, **kwargs)

    self.update_path()

  def update_path(self):
    xlimits = self.ax.get_xlim()
    self.xpoints = np.linspace(xlimits[0], xlimits[1], self._samples)
    self.line.set_data(self.xpoints, list(map(self._function, self.xpoints)))

  @property
  def function(self):
    return self._function

  @function.setter
  def function(self, function):
    self._function = function
    self.update_path()

  @property
  def samples(self):
    return self._samples

  @samples.setter
  def samples(self, samples):
    self._samples = samples
    self.update_path()

In [319]:
class LineFunctionObject(FunctionObject):
  def __init__(self, plot, function, **kwargs):
    super().__init__(plot, function, samples=2, **kwargs)

In [320]:
class AI():
  def __init__(self, w=1, b=0):
    self.w = w
    self.b = b

  @staticmethod
  def predict(x, w=1, b=0):
    return w * x + b

  def _instance_predict(self, x):
    return self.predict(x, self.w, self.b)

  @staticmethod
  def descent_gradient(data, w=1, b=0):
    squaredSum = 0
    gradient = 0

    for x, y in data:
      diff = y - AI.predict(x, w, b)
      squaredSum += diff**2
      gradient += -2 * diff

    return squaredSum, gradient

  def _instance_descent_gradient(self, data):
    return self.descent_gradient(data, self.w, self.b)

  def learn(self, data, learn_rate=0.1, max_reps=1000, snapshots=False):
    squaredSum, loss = 0, 0
    for _ in range(max_reps):
      squaredSum, loss = self._instance_descent_gradient(data)

      if snapshots:
        yield self.b, squaredSum, loss

      if loss == 0:
        break

      self.b -= learn_rate * loss

In [321]:
DATA = [
  (0.5, 1.4),
  (2.3, 1.9),
  (2.9, 3.2)
]

In [329]:
def visually_train(data, learn_rate=0.1, max_reps=100):
  ai = AI(w=0.64)
  plot = Plot(grid=(1, 2), figsize=(8, 4), axis=[(0, 3, 0, 4), (-1, 3, 0, 8)])

  # Setup left
  data_lines = [LineObject(plot.ax[0], point, (point[0], ai.predict(point[0]))) for point in DATA]
  data_points = [PointObject(plot.ax[0], point) for point in DATA]
  predict_line = LineFunctionObject(plot.ax[0], ai._instance_predict, color='lightblue')

  # Setup right
  loss_function = FunctionObject(plot.ax[1], lambda b: AI.descent_gradient(DATA, ai.w, b)[0], color='blue')
  gradient_line = LineFunctionObject(plot.ax[1], lambda b: -1, color='green')

  # Setup training
  train_iterations = list(ai.learn(DATA, learn_rate=learn_rate, max_reps=max_reps, snapshots=True))

  def update(frame):
    b, squaredSum, loss = train_iterations[frame]

    # Updating left
    for data_line in data_lines:
      data_line.end = (data_line.start[0], AI.predict(data_line.start[0], ai.w, b))
    predict_line.function = lambda x: AI.predict(x, ai.w, b)

    # Updating right
    gradient_line.function = lambda x: loss * (x - b) + squaredSum
    p = PointObject(plot.ax[1], (b, squaredSum), color='red')

    return *map(lambda dl: dl.line, data_lines), predict_line.line, gradient_line.line, p.line

  return plot.animate(update, frames=len(train_iterations))

In [337]:
visually_train(DATA, 0.01)